Тестовое задание

Цель: проверить гипотезу по уточнению долгосрочного прогноза

Гипотеза: на основании среднего смещения любого из параметров за предыдущий месяц от текущей даты, смещение средних многолетних значений на год вперёд от текущей даты окажется ближе к фактическим значениям, чем средние многолетние значения. Иными словами: на основании поведения параметра за последний месяц (в среднем теплее, холоднее), может быть дано уточнение к долгосрочному прогнозу на основе среднемноголетних (что такая коррекция будет обладать более высокой точностью, чем просто среднее многолетнее значение)

Задача: на основе фактических значений с метеостанции Самара проверить данную гипотезу для нескольких дат по температуре и осадкам. Для этого в ретроспективе взять несколько любых опорных дат и сверять для них значения долгосрочного прогноза, построенного на основе этой гипотезы, и фактические значения параметров. Использовать только столбцы B и D (это температура воздуха и влажность воздуха). Расхождения между рассчитанным значением и фактом охарактеризовать численно.


Подключение библиотек

In [ ]:
import pandas as pd
import numpy as np
import calendar
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

Чтение данных. Используем только колонки с датой, температурой и влажностью

In [ ]:
data = pd.read_excel('data.xls', usecols='A,B,D', names=['date', 'temperature', 'humidity'])

data.head()

Проверяем наличие пропущенных значений в таблице

In [ ]:
data.isna().mean()

Напишем функцию, которая будет заполнять пустые ячейки с помощью линейной интерполяции

In [ ]:
def fill_missing_cells(column):
    return column.interpolate(method='linear', limit_direction='forward')
         

Применяем функцию к столбцам, где есть пропущенные значения, и проверяем, что пустых ячеек нет

In [ ]:
data['temperature'] = fill_missing_cells(data['temperature'])
data['humidity'] = fill_missing_cells(data['humidity'])

data.isna().mean()

Преобразуем данные в столбце date к типу datetime

In [ ]:
data['date'] = pd.to_datetime(data['date'], format='%d.%m.%Y %H:%M')

data['date'].dtype

После того, как данные подготовлены, можно приступать к проверке гипотезы. Для начала выберем несколько опорных дат. Возьмем таких 100 случайных значений, что для каждой даты есть данные за как минимум два предыдущих года и данные для того же дня на следующий год, чтобы проверить расхождение

In [ ]:
min_date = data['date'].min()
max_date = data['date'].max()

data_for_reference = data[(data['date'] > min_date + pd.DateOffset(years=2)) & (data['date'] < max_date - pd.DateOffset(years=1))]

reference_dates = pd.Series(np.random.choice(data_for_reference['date'], size=100, replace=False))
test_dates = reference_dates + pd.DateOffset(years=1)

Напишем функцию расчета среднесуточного значения заданного параметра 

In [ ]:
def calculate_daily_avg(date, param):
    daily_data = data[data['date'].dt.date == date.date()] 
    return daily_data[param].mean()
    

Напишем функцию, которая считает среднесуточные значения за предыдущий месяц

In [ ]:
def calculate_daily_avg_for_last_month(date, param):
    end_date = date.date()
    start_date = end_date - pd.DateOffset(days=30)
    date_range = pd.date_range(start=start_date, end=end_date)
    daily_averages = [calculate_daily_avg(single_date, param) for single_date in date_range]
    return pd.Series(daily_averages)

Напишем функцию, которая считает среднее многолетнее значение среднесуточного значения параметра

In [ ]:
def calculate_long_term_avg(date, param):
    if calendar.isleap(int(date.year)) and date.day == 29 and date.month == 2:
        date = date - pd.DateOffset(days=1)
    day_of_years = []
    for year in range(min_date.year, date.year + 1):
        new_date = datetime(year, date.month, date.day)
        if min_date <= new_date <= date:
            day_of_years.append(new_date)
    yearly_values = [calculate_daily_avg(single_date, param) for single_date in day_of_years]
    return pd.Series(yearly_values).mean()

Напишем функцию, которая считает среднее всех отклонений между среднесуточным за месяц и средним многолетним и корректирует прогноз на основе среднемноголетних

In [ ]:
def adjusted_long_term_forecast(date, param):
    daily_avg_for_last_month = calculate_daily_avg_for_last_month(date, param)
    long_term_avg = calculate_long_term_avg(date, param)
    deviation = (daily_avg_for_last_month - long_term_avg).mean()
    return long_term_avg + deviation

Считаем прогноз без корректировки

In [ ]:
long_term_forecast_temp = pd.Series([calculate_long_term_avg(date, 'temperature') for date in reference_dates])
long_term_forecast_hum = pd.Series([calculate_long_term_avg(date, 'humidity') for date in reference_dates])

Считаем прогноз с корректировкой 

In [ ]:
adjusted_long_term_forecast_temp = pd.Series([adjusted_long_term_forecast(date, 'temperature') for date in reference_dates])
adjusted_long_term_forecast_hum = pd.Series([adjusted_long_term_forecast(date, 'humidity') for date in reference_dates])

Считаем реальные среднесуточные значения

In [ ]:
actual_dat_temp = pd.Series([calculate_daily_avg(date, 'temperature') for date in test_dates])
actual_dat_hum = pd.Series([calculate_daily_avg(date, 'humidity') for date in test_dates])

Таблица с температурой

In [ ]:
result_temp = pd.DataFrame({'Date':test_dates, 'Actual data': actual_dat_temp, 'The average long-term forecast': long_term_forecast_temp, 
                            'Adjusted long-term average forecast': adjusted_long_term_forecast_temp})
result_temp 

Таблица с влажностью

In [ ]:
result_hum = pd.DataFrame({'Date':test_dates, 'Actual data': actual_dat_hum, 'The average long-term forecast': long_term_forecast_hum, 
                            'Adjusted long-term average forecast': adjusted_long_term_forecast_hum})
result_hum 

Построим графики 

In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(result_temp['Actual data'], label='Actual data')  
plt.plot(result_temp['The average long-term forecast'], label='The average long-term forecast')  
plt.plot(result_temp['Adjusted long-term average forecast'], label='Adjusted long-term average forecast') 
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(result_hum['Actual data'], label='Actual data')  
plt.plot(result_hum['The average long-term forecast'], label='The average long-term forecast')  
plt.plot(result_hum['Adjusted long-term average forecast'], label='Adjusted long-term average forecast') 
plt.legend()

plt.show()

Посчитаем ошибку по температуре

In [ ]:
mae_avg_long_term_forecast = mean_absolute_error(result_hum['Actual data'], result_hum['The average long-term forecast'])
mse_avg_long_term_forecast = mean_squared_error(result_hum['Actual data'], result_hum['The average long-term forecast'])

mae_avg_long_term_forecast_adj = mean_absolute_error(result_hum['Actual data'], result_hum['Adjusted long-term average forecast'])
mse_avg_long_term_forecast_adj = mean_squared_error(result_hum['Actual data'], result_hum['Adjusted long-term average forecast'])

print(f'MAE: The average long-term forecast - {mae_avg_long_term_forecast}, Adjusted long-term average forecast - {mae_avg_long_term_forecast_adj}')
print(f'MSE: The average long-term forecast - {mse_avg_long_term_forecast}, Adjusted long-term average forecast - {mse_avg_long_term_forecast_adj}')

Посчитаем ошибку по влажности

In [ ]:
mae_avg_long_term_forecast = mean_absolute_error(result_temp['Actual data'], result_temp['The average long-term forecast'])
mse_avg_long_term_forecast = mean_squared_error(result_temp['Actual data'], result_temp['The average long-term forecast'])

mae_avg_long_term_forecast_adj = mean_absolute_error(result_temp['Actual data'], result_temp['Adjusted long-term average forecast'])
mse_avg_long_term_forecast_adj = mean_squared_error(result_temp['Actual data'], result_temp['Adjusted long-term average forecast'])

print(f'MAE: The average long-term forecast - {mae_avg_long_term_forecast}, Adjusted long-term average forecast - {mae_avg_long_term_forecast_adj}')
print(f'MSE: The average long-term forecast - {mse_avg_long_term_forecast}, Adjusted long-term average forecast - {mse_avg_long_term_forecast_adj}')

Из проделанной работы можно сделать вывод, что уточнение к долгосрочному прогнозу на основе среднемноголетних значений даёт небольшое улучшении при предсказании температуры и значительно ухудшает результаты при предсказании влажности